In [1]:
import numpy as np # library to handle data in a vectorized manner
!pip3 install --upgrade numpy

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 

import requests 
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Requirement already up-to-date: numpy in /Applications/anaconda3/lib/python3.7/site-packages (1.16.0)
Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### I read the csv file with all Toronto Postcode coordinates and convert it to a pandas dataframe

In [2]:
toronto = pd.read_csv('toronto_co.csv')

In [57]:
toronto.rename(columns={'Neighbourhood':'Neighborhood'}, inplace = True)
print(toronto.columns)

Index(['Unnamed: 0', 'Postcode', 'Borough', 'Neighborhood', 'Latitude',
       'Longitude'],
      dtype='object')


In [90]:
toronto.drop(['Unnamed: 0','Postcode'], axis = 1, inplace = True)

In [91]:
toronto.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


#### I find the coordinates of Toronto

In [92]:
address = 'Toronto, Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude_x,longitude_y))


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geographical coordinates of Toronto are 43.653963, -79.387207.


### I create the map of Toronto

In [161]:
map_toronto = folium.Map(location=[latitude_x,longitude_y], zoom_start=10)
map_toronto

### I insert the markers with the Neighbours

In [162]:
for lat, lng, bor, nei in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'],toronto['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'red',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity=0.7,
        parse_html = False).add_to(map_toronto)

map_toronto

In [95]:
CLIENT_ID = 'HMN0ROLECZHKTAGFQS4KCTMWDWX5FPCWEMWM5AABQNRAV5LN' 
CLIENT_SECRET = 'BYA5TUW2DOC02Z5NZZ5RLUPOAGWGKFQCLL4CFWMEI2VZIKUB'
VERSION = '20180609' 
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HMN0ROLECZHKTAGFQS4KCTMWDWX5FPCWEMWM5AABQNRAV5LN
CLIENT_SECRET:BYA5TUW2DOC02Z5NZZ5RLUPOAGWGKFQCLL4CFWMEI2VZIKUB


### I explore the second neighborhood in my dataframe

In [96]:
toronto.loc[1,'Neighborhood']

'Port Union,Rouge Hill,Highland Creek'

In [97]:
nei_lat = toronto.loc[1,'Latitude']
nei_lon = toronto.loc[1,'Longitude']
nei_name = toronto.loc[1,'Neighborhood']
print('Latitude and longitude of {} are {} and {}'.format(nei_name, nei_lat, nei_lon))

Latitude and longitude of Port Union,Rouge Hill,Highland Creek are 43.7845351 and -79.16049709999999


### I look for the 100 venues that are in Hillcrest Village in a radius of 500 meters

In [98]:
LIMIT = 100

In [99]:
radius = 500

In [100]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    nei_lat, 
    nei_lon, 
    radius, 
    LIMIT)

url



'https://api.foursquare.com/v2/venues/explore?&client_id=HMN0ROLECZHKTAGFQS4KCTMWDWX5FPCWEMWM5AABQNRAV5LN&client_secret=BYA5TUW2DOC02Z5NZZ5RLUPOAGWGKFQCLL4CFWMEI2VZIKUB&v=20180609&ll=43.7845351,-79.16049709999999&radius=500&limit=100'

In [106]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c44a77fdb04f57d55662d1c'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.7890351045, 'lng': -79.15427558741125},
   'sw': {'lat': 43.7800350955, 'lng': -79.16671861258872}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c23d3aaf7ced13a5ed7216d',
       'name': 'Royal Canadian Legion',
       'location': {'address': '45 Lawson Rd',
        'lat': 43.78253332838298,
        'lng': -79.16308473261682,
        'labeledLatLngs': [{'label': 'display',
          'lat': 4

In [107]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']



In [108]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


,name,categories,lat,lng
0,Royal Canadian Legion,Bar,43.782533,-79.163085
1,Affordable Toronto Movers,Moving Target,43.787919,-79.162977


In [109]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [163]:
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )


Malvern,Rouge
Port Union,Rouge Hill,Highland Creek
Morningside,West Hill,Guildwood
Woburn
Cedarbrae
Scarborough Village
Kennedy Park,East Birchmount Park,Ionview
Clairlea,Oakridge,Golden Mile
Scarborough Village West,Cliffside,Cliffcrest
Cliffside West,Birch Cliff
Dorset Park,Wexford Heights,Scarborough Town Centre
Wexford,Maryvale
Agincourt
Sullivan,Clarks Corners,Tam O'Shanter
Milliken,Agincourt North,Steeles East,L'Amoreaux East
Steeles West,L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Oriole,Henry Farm
Bayview Village
Silver Hills,York Mills
Willowdale,Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South,Flemingdon Park
Downsview North,Wilson Heights,Bathurst Manor
York University,Northwood Park
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
India Ba

In [ ]:
print(toronto_venues.shape)

In [111]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern,Rouge",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Morningside,West Hill,Guildwood",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Morningside,West Hill,Guildwood",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [112]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Albion Gardens,Thistletown,South Steeles,Beaumond Heights,Jamestown,Humbergate,Silverstone,Mount Olive",10,10,10,10,10,10
"Alderwood,Long Branch",10,10,10,10,10,10
Bayview Village,4,4,4,4,4,4
Berczy Park,54,54,54,54,54,54
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CFB Toronto,Downsview East",4,4,4,4,4,4
"CN Tower,South Niagara,Harbourfront West,Island airport,King and Spadina,Railway Lands,Bathurst Quay",14,14,14,14,14,14


In [74]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 276 unique categories.


In [75]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

neigh = toronto_onehot['Neighborhood']
toronto_onehot.drop(labels=['Neighborhood'], axis = 1, inplace = True)
toronto_onehot.insert(0, 'Neighborhood',neigh)
toronto_onehot.head() 



,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Malvern,Rouge",0,0,0,0,0,0,0,0,

In [76]:
toronto_onehot.shape

(2252, 276)

In [35]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.00000

In [36]:
toronto_grouped.shape

(99, 276)

In [77]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3      Thai Restaurant  0.04
4           Steakhouse  0.04


----Agincourt----
               venue  freq
0     Sandwich Place  0.25
1     Breakfast Spot  0.25
2             Lounge  0.25
3       Skating Rink  0.25
4  Mobile Phone Shop  0.00


----Albion Gardens,Thistletown,South Steeles,Beaumond Heights,Jamestown,Humbergate,Silverstone,Mount Olive----
                  venue  freq
0         Grocery Store   0.2
1          Liquor Store   0.1
2              Pharmacy   0.1
3           Pizza Place   0.1
4  Fast Food Restaurant   0.1


----Alderwood,Long Branch----
            venue  freq
0     Pizza Place   0.2
1             Gym   0.1
2    Skating Rink   0.1
3  Sandwich Place   0.1
4    Dance Studio   0.1


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2   Chinese Restaurant  0.25
3 

                  venue  freq
0                  Park  0.10
1           Pizza Place  0.05
2    Italian Restaurant  0.05
3  Fast Food Restaurant  0.05
4          Burger Joint  0.05


----Islington,Martin Grove,West Deane Park,Cloverdale,Princess Gardens----
               venue  freq
0               Bank   0.5
1        Golf Course   0.5
2  Accessories Store   0.0
3        Music Store   0.0
4      Moving Target   0.0


----Kennedy Park,East Birchmount Park,Ionview----
                venue  freq
0      Discount Store  0.33
1         Coffee Shop  0.17
2    Department Store  0.17
3  Chinese Restaurant  0.17
4       Train Station  0.17


----Kensington Market,Grange Park,Chinatown----
                           venue  freq
0                           Café  0.07
1                            Bar  0.07
2  Vegetarian / Vegan Restaurant  0.05
3                    Coffee Shop  0.04
4            Dumpling Restaurant  0.04


----Kingsway Park South East,Old Mill South,Humber Bay,Royal York South Eas

                        venue  freq
0                 Coffee Shop   0.4
1        Gym / Fitness Center   0.2
2                         Pub   0.2
3           Accessories Store   0.0
4  Modern European Restaurant   0.0


----The Danforth West,Riverdale----
                venue  freq
0    Greek Restaurant  0.24
1         Coffee Shop  0.10
2      Ice Cream Shop  0.07
3  Italian Restaurant  0.05
4           Bookstore  0.05


----The Junction South,High Park----
                  venue  freq
0                  Café  0.09
1    Mexican Restaurant  0.09
2  Fast Food Restaurant  0.04
3             Gastropub  0.04
4                   Bar  0.04


----Thorncliffe Park----
                  venue  freq
0     Indian Restaurant  0.12
1  Gym / Fitness Center  0.06
2           Supermarket  0.06
3        Sandwich Place  0.06
4          Liquor Store  0.06


----Toronto Dominion Centre,Design Exchange----
                 venue  freq
0          Coffee Shop  0.14
1                Hotel  0.08
2              

### Display the 10 most common venues for each Neighborhood

In [113]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [114]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Restaurant,Gym,Clothing Store,Bakery,Asian Restaurant
1,Agincourt,Lounge,Sandwich Place,Skating Rink,Breakfast Spot,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Albion Gardens,Thistletown,South Steeles,Beaum...",Grocery Store,Pizza Place,Coffee Shop,Sandwich Place,Liquor Store,Beer Store,Fried Chicken Joint,Pharmacy,Fast Food Restaurant,Empanada Restaurant
3,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Skating Rink,Sandwich Place,Pub,Pool,Dance Studio,Pharmacy,Gym,Dumpling Restaurant
4,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
5,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Café,Steakhouse,Italian Restaurant,Beer Bar,Seafood Restaurant,Farmers Market,Pub
6,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Garden Center,Skate Park,Park,Farmers Market,Fast Food Restaurant,Brewery,Spa,Light Rail Station,Burrito Place
7,"CFB Toronto,Downsview East",Bus Stop,Playground,Airport,Park,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
8,"CN Tower,South Niagara,Harbourfront West,Islan...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boat or Ferry,Sculpture Garden,Plane,Boutique,Airport Gate,Airport
9,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Café,Pizza Place,Bakery,Italian Restaurant,Pub,Market,Diner,Snack Place


### Cluster neighbohoods


In [151]:
kclusters = 5

toronto_grouped_cluster = toronto_grouped.drop('Neighborhood',1)

kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(toronto_grouped_cluster)

kmeans.labels_[0:10]



array([0, 0, 0, 0, 0, 0, 0, 4, 0, 0], dtype=int32)

In [123]:
toronto.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [140]:
toronto_merged = toronto




In [143]:
toronto_merged.shape

(99, 4)

In [121]:
kmeans.labels_.shape

(99,)

In [142]:
toronto_merged.tail(5)

,Borough,Neighborhood,Latitude,Longitude
94,Etobicoke,"Islington,Martin Grove,West Deane Park,Cloverd...",43.650943,-79.554724
95,Etobicoke,"Eringate,Old Burnhamthorpe,Markland Wood,Bloor...",43.643515,-79.577201
96,North York,Humber Summit,43.756303,-79.565963
97,North York,"Emery,Humberlea",43.724766,-79.532242
98,York,Weston,43.706876,-79.518188


In [144]:



toronto_merged['Cluster Labels'] = kmeans.labels_


toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Malvern,Rouge",43.806686,-79.194353,0,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
1,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497,0,Moving Target,Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Diner
2,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711,0,Pizza Place,Electronics Store,Rental Car Location,Breakfast Spot,Mexican Restaurant,Medical Center,Empanada Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dim Sum Restaurant
3,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Pharmacy,Korean Restaurant,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
4,Scarborough,Cedarbrae,43.773136,-79.239476,0,Lounge,Thai Restaurant,Bank,Bakery,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Hakka Restaurant,Coworking Space,Doner Restaurant


In [149]:
import folium

# create map
map_clusters = folium.Map(location=[latitude_x, longitude_y], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### CLUSTER 1 - RESTAURANTS

In [154]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern,Rouge",Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
1,"Port Union,Rouge Hill,Highland Creek",Moving Target,Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Diner
2,"Morningside,West Hill,Guildwood",Pizza Place,Electronics Store,Rental Car Location,Breakfast Spot,Mexican Restaurant,Medical Center,Empanada Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dim Sum Restaurant
3,Woburn,Coffee Shop,Pharmacy,Korean Restaurant,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
4,Cedarbrae,Lounge,Thai Restaurant,Bank,Bakery,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Hakka Restaurant,Coworking Space,Doner Restaurant
5,Scarborough Village,Playground,Construction & Landscaping,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
6,"Kennedy Park,East Birchmount Park,Ionview",Discount Store,Department Store,Coffee Shop,Chinese Restaurant,Train Station,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Dim Sum Restaurant
8,"Scarborough Village West,Cliffside,Cliffcrest",Intersection,Motel,American Restaurant,Dim Sum Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
9,"Cliffside West,Birch Cliff",College Stadium,General Entertainment,Skating Rink,Café,Concert Hall,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
11,"Wexford,Maryvale",Smoke Shop,Breakfast Spot,Shopping Mall,Bakery,Auto Garage,Sandwich Place,Yoga Studio,Dumpling Restaurant,Doner Restaurant,Donut Shop


### CLUSTER 2 AIRPORT

In [155]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,"CFB Toronto,Downsview East",Bus Stop,Playground,Airport,Park,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
44,Lawrence Park,Bus Line,Swim School,Park,Dim Sum Restaurant,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop


### CLUSTER 3 RESTAURANTS

In [156]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Rosedale,Park,Playground,Trail,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
62,"Lawrence Manor East,Bedford Park",Italian Restaurant,Juice Bar,Fast Food Restaurant,Sushi Restaurant,Coffee Shop,Pharmacy,Pizza Place,Restaurant,Butcher,Café


### CLUSTER 4 RIVER

In [158]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"York University,Northwood Park",Bar,Massage Studio,Coffee Shop,Furniture / Home Store,Miscellaneous Shop,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore
90,"Montgomery Road,The Kingsway,Old Mill North",River,Park,Pool,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


### CLUSTER 5 RESTAURANTS AND SHOPS

In [160]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,"Clairlea,Oakridge,Golden Mile",Bakery,Bus Line,Fast Food Restaurant,Intersection,Metro Station,Park,Soccer Field,Bus Station,Event Space,Ethiopian Restaurant
10,"Dorset Park,Wexford Heights,Scarborough Town C...",Indian Restaurant,Pet Store,Vietnamese Restaurant,Latin American Restaurant,Furniture / Home Store,Chinese Restaurant,Drugstore,Diner,Discount Store,Dog Run
36,Woodbine Heights,Skating Rink,Athletics & Sports,Video Store,Curling Ice,Spa,Cosmetics Shop,Beer Store,Park,Pharmacy,Event Space
39,Thorncliffe Park,Indian Restaurant,Yoga Studio,Supermarket,Grocery Store,Liquor Store,Park,Pharmacy,Gym / Fitness Center,Pizza Place,Coffee Shop
47,Davisville,Sandwich Place,Pizza Place,Dessert Shop,Café,Italian Restaurant,Seafood Restaurant,Sushi Restaurant,Coffee Shop,Pharmacy,Chinese Restaurant
51,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Café,Pizza Place,Bakery,Italian Restaurant,Pub,Market,Diner,Snack Place
52,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Fast Food Restaurant,Mediterranean Restaurant,Café,Pub
54,"Garden District,Ryerson",Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Ramen Restaurant,Diner,Restaurant,Tea Room,Thai Restaurant
57,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Spa,Sandwich Place,Salad Place,Bubble Tea Shop,Ice Cream Shop
66,"University of Toronto,Harbord",Café,Coffee Shop,Bar,Bookstore,Restaurant,Bakery,Japanese Restaurant,Nightclub,Noodle House,Comfort Food Restaurant
